In [1]:
ls hl-node-fills/

In [ ]:
ls hl-node-trades/20250525/

In [ ]:
!realpath hl-node-trades/20250525

In [ ]:
import os
import ujson as json
import pandas as pd

import lz4.frame
from loguru import logger

In [ ]:
BASE_DIR = os.getcwd()

old_data_dates = os.listdir(os.path.join(BASE_DIR, "hl-node-trades"))

In [ ]:
def retrieve_data(file_path):
    records = []

    with open(file_path) as f:
        append = records.append
        for line in f:
            trade = json.loads(line)
            for side_info, side in zip(trade.get("side_info"), ["A", "B"]):
                append(
                    {
                        "coin": trade.get("coin"),
                        "price": float(trade.get("px")),
                        "size": float(trade.get("sz")),
                        "time": trade.get("time"),
                        "side": side,
                        "wallet": side_info.get("user"),
                    }
                )

    df = pd.DataFrame.from_records(records)
    df["time"] = pd.to_datetime(df["time"])
    return df


def filter_data(df, coin):
    return df[df["coin"] == coin]


def save_data(df, output_file):
    # Write to disk
    if not df.empty:
        df.to_parquet(output_file, index=False, engine='pyarrow', compression='snappy', append=os.path.exists(output_file))
        logger.info("Data has been saved successfully.")
    else:
        logger.warnning("No data to save.")


In [ ]:
coin = "ETH"
old_data_folders = os.listdir(os.path.join(BASE_DIR, "hl-node-trades"))

for date in old_data_folders:
    hour_file_names = os.listdir(os.path.join(BASE_DIR, "hl-node-trades", date))

    for file_name in hour_file_names:
        file_full_path = os.path.join(BASE_DIR, "hl-node-trades", date, file_name)
        result_full_path = os.path.join(BASE_DIR, "hl-node-trades", coin)

        df = retrieve_data(file_full_path)
        df = filter_data(df, coin)
        save_data(df, result_full_path)

    break